In [1]:
import numpy as np
import pandas as pd
import sys
import os
import fur.path_assistant as path_assistant
shift = path_assistant.PathAssistant('shift_03_16_2020',
    ignore_files=['RefCurve_2020-03-16_0_084020_test.Wfm.bin',
                  'RefCurve_2020-03-16_153_155200.Wfm.bin'])
from config import get_from_config, save_to_config
from scipy.interpolate import interp1d
from lattice.summary_in_undulator import get_summary_in_undulator
lattice_file = shift.get_6dsim_dir().fi("IOTA_1NL_100MeV_v8.6.1.4.6ds_data.txt")

In [2]:
und_summary = get_summary_in_undulator(
    lattice_file,
    np.ones(8),
    np.ones(8),
    0,
    0
    )
Dx, betax, betay, alphax, alphay = und_summary[0].loc['Middle', ['Dispersion_cm_X', 'Beta_cm_X', 'Beta_cm_Y',
                          'Alpha_X', 'Alpha_Y']]
Dx, betax, betay, alphax, alphay

/home/ilobach/fur/env/lib/python3.6/site-packages/pandas/core/series.py:679: RuntimeWarning: invalid value encountered in sqrt
  result = getattr(ufunc, method)(*inputs, **kwargs)
/home/ilobach/fur/lattice/lattice.py:160: RuntimeWarning: invalid value encountered in sqrt
  return np.sqrt(e_um*1e4*beta_cm+(1e4*dispersion_cm*dpp)**2)


(101.20460727716554,
 203.63015539385736,
 98.4141815632213,
 1.2502870273577453,
 -0.8742874700127816)

In [3]:
def newval(name, val, unit):
    return (r'\newcommand{\val'+name+r'}{\SI{'
              +val
              +r'}{'
              +unit
              +r'}}'+'\n')

In [18]:
cur_to_sum_channel = get_from_config("Beam_current_to_Sum_channel_ampl_V/mA")
sum_channel_to_photoelectrons =\
    get_from_config('sum_channel_to_photoelectrons')
meas_photons_per_electron =\
    get_from_config("Measured_photons_per_electron")
meas_photons_per_electron
cspeed = get_from_config("c_m/s")
Ibeam = get_from_config("NominalBeamCurrent_mA")
N_to_I = 1/sum_channel_to_photoelectrons/cur_to_sum_channel
N0 = Ibeam/N_to_I
Ku = get_from_config("K_peak")
gamma = get_from_config("gamma")
Ebeam = 0.511*gamma
Ku_error = get_from_config("Ku_error")
Ebeam_error= get_from_config("Ebeam_error_MeV")
und_per = get_from_config("Undulator_period_m")
lambda1 = und_per/2/gamma**2*(1+Ku**2/2)*1e6
zobs = get_from_config("z_obs_m")
inch = 0.0254 # m
apx_mrad = 1e3*inch/zobs
apy_mrad = apx_mrad/np.sqrt(2)
integrator_to_photoelectrons =\
    get_from_config("integrator_amp_to_photoelectrons")
hybrid_balance_error = get_from_config("HybridBalanceError")
hybrid_errorbar = 2*hybrid_balance_error
xi = get_from_config("integrator_amp_to_photoelectrons")\
    /get_from_config("sum_channel_to_photoelectrons")/2
test_light_source_theta = get_from_config("test_light_source_theta")
noise_level = get_from_config("noise_level")
noise_level_V2 = get_from_config("noise_level_V2")
varN_error_fit = get_from_config("varN_error_fit")

In [5]:
res_df_loaded = \
    pd.read_csv(shift.get_results_dir().fi('meas_ROUND_03_16_2020.csv'),
                index_col=0)
res_df_ROUND = res_df_loaded[res_df_loaded['N']<2.25e7]
res_df_loaded = \
    pd.read_csv(shift.get_results_dir().fi('meas_FLAT_03_16_2020.csv'),
                index_col=0)
res_df_FLAT = res_df_loaded[res_df_loaded['N']<2.4e7]
MrMf = []
for res_df in [res_df_ROUND, res_df_FLAT]:
    res_df = res_df.sort_values(by='N')
    varN = np.interp(N0, res_df['N'], res_df['varN'])
    M = N0**2/(varN-N0)
    MrMf.append(M)
Mr, Mf = MrMf

## Re-calculate predicted photons per electron

In [6]:
# from wiggler_radiation.Wigrad.wigrad_generator import \
#     generate_wr_sim_with_wigrad_results
# wr_sim = generate_wr_sim_with_wigrad_results(
# K_peak_in=Ku,
# gamma_in=gamma)
# ampx3d = wr_sim.get_amplittude_3D(polarization='x')
# mesh = get_from_config("radiation_mesh")
# xmin, xmax, _ = mesh[0]
# xmin = xmin/zobs
# xmax = xmax/zobs
# ymin, ymax, _ = mesh[1]
# ymin = ymin/zobs
# ymax = ymax/zobs
# lmin, lmax, _ = mesh[2]
# dax = (xmax-xmin)/(mesh[0][2]-1)
# day = (ymax-ymin)/(mesh[1][2]-1)
# dl = (lmax-lmin)/(mesh[2][2]-1)
# tot =dax*day*dl*np.sum(np.absolute(ampx3d)**2)
# save_to_config("Predicted_photons_per_electron_wigrad", tot)

In [7]:
photonFluxMeas = get_from_config("Measured_photons_per_electron")
photonFluxTheor = get_from_config("Predicted_photons_per_electron_wigrad")

In [8]:
# Vertical size and emittance of flat beam
meas_FLAT = pd.read_csv(
    shift.get_results_dir().fi('meas_FLAT_03_16_2020.csv'), index_col=0)
meas_FLAT = meas_FLAT.sort_values('N')
sy_flat = np.interp(N0, meas_FLAT['N'], meas_FLAT['sy_rec'])
ey_flat = np.interp(N0, meas_FLAT['N'], meas_FLAT['ey_rec'])

In [9]:
Nu = 10.5

## zero-current 

In [10]:
iota_period_sec = get_from_config("IOTA_revolution_period")
c = 2.9979e10
gamma = get_from_config("gamma")
hbar_c = 0.1973269804e-4
rho = 70
Ec = 3/2*hbar_c*gamma**3/rho
Et = 88.9e-7*(gamma*0.511)**4/rho
Scale = 15*np.sqrt(3)/8*Et
alpha = 0.07088
V = 380
E0 = gamma*511000
f = 30e6
h = 4
v0 = V/E0
S = Scale/E0
eta = alpha - 1/gamma**2
w = 2*np.pi*h*eta
ts = 0.836
JE = 2*E0*iota_period_sec/ts/Et
delta_rms = 0.62e-6*gamma/np.sqrt(JE*rho/100)
tau_rms = 1e9*delta_rms/(f*2*np.pi*np.sqrt(V/(2*np.pi*E0*h*np.abs(eta))))
sz_zero_cur = tau_rms*1e-9*c

In [11]:
nrev = 7.5e6*1.5e-3

In [19]:
res = \
    r'% Do not edit this file. It is generated by a Python script.'+'\n'\
    +r'\ProvidesPackage{numericalvalues}'+'\n'\
    +r'\usepackage{siunitx}'+'\n'+'\n'+'\n'

res += newval("varNerror", f"{varN_error_fit:.1e}", r"")
res += newval("noiseLevel", f"{noise_level:.1e}", r"")
res += newval("noiseLevelV", f"{noise_level_V2:.1e}", r"V^2")
res += newval("theta", f"{test_light_source_theta:.2e}", r"")
res += newval("thetarms", f"{np.sqrt(test_light_source_theta):.2e}", r"")
res += newval("xi", f"{xi:.2f}", r"")
res += newval("Dx", f"{Dx:.0f}", r"cm")
res += newval("betax", f"{betax:.0f}", r"cm")
res += newval("betay", f"{betay:.0f}", r"cm")
res += newval("alphax", f"{alphax:.2f}",r'')
res += newval("alphay", f"{alphay:.2f}",r'')
res += newval("szZeroCur", f"{sz_zero_cur:.0f}", r"cm" )
res += newval("spZeroCur", f"{delta_rms:.1e}", r'')
res += newval("wfLen", "1.5", "ms")
res += newval("Nrev", f"{nrev:.0f}", "")
res += newval("EbeamDimLess", f"{Ebeam:.1f}", "")
res += newval("Ebeam", f"{Ebeam:.1f}", 'MeV')
res += newval("Ku", f"{Ku:.1f}", '')
res += newval("lambdaone", f"{lambda1:.2f}", r'\micro m')
res += newval("Ibeam", f"{Ibeam:.1f}", 'mA')
res += newval("Mround", f"{Mr:.1e}", '')
res += newval("Mflat", f"{Mf:.1e}", '')
res += newval("zobs", f"{zobs:.1f}", 'm')
res += newval("semiApertureX", f"{apx_mrad:.1f}", 'mrad')
res += newval("semiApertureY", f"{apy_mrad:.1f}", 'mrad')
res += newval("oneOverGammaNu", f"{1/gamma/np.sqrt(Nu)*1e3:.1f}", 'mrad')
res += newval("photonFluxMeas", f"{photonFluxMeas:.1e}", 'photoelectrons/electron')
res += newval("photonFluxTheor", f"{photonFluxTheor:.1e}", 'photoelectrons/electron')
res += newval("AmpToPhotoel", f"{integrator_to_photoelectrons:.2e}",
              "photoelectrons/V")
res += newval("Kuerror", f"{Ku_error:.2f}", '')
res += newval("Ebeamerror", f"{Ebeam_error:.0f}", "MeV")
res += newval("HybridErrorbar", f"{hybrid_errorbar*100:.0f}",r'\percent')
res += newval("NundPer", f"{Nu:.1f}", "")


for kind in ["round", "flat"]:
    res+='\n'+r'% '+kind+" beam parameters"+'\n'
    bp_df = pd.read_csv(
        shift.get_results_dir().fi(f"beam_params_vs_current_{kind}.csv"))
    sx_func = interp1d(bp_df["N"], bp_df['Sigma_um_X'],
                       bounds_error=False, fill_value="extrapolate")
    sy_func = interp1d(bp_df["N"], bp_df['Sigma_um_Y'],
                       bounds_error=False, fill_value="extrapolate")
    sz_func = interp1d(bp_df["N"], bp_df['sz_um'],
                       bounds_error=False, fill_value="extrapolate")
    ex_func = interp1d(bp_df["N"], bp_df['ex_um'],
                       bounds_error=False, fill_value="extrapolate")
    ey_func = interp1d(bp_df["N"], bp_df['ey_um'],
                       bounds_error=False, fill_value="extrapolate")
    dpp_func = interp1d(bp_df["N"], bp_df['dp/p'],
                        bounds_error=False, fill_value="extrapolate")
    cspeed = get_from_config("c_m/s")
    Ibeam = get_from_config("NominalBeamCurrent_mA")
    N_to_I = 1/sum_channel_to_photoelectrons/cur_to_sum_channel
    sx = sx_func(N0)
    sy = sy_func(N0) if (kind == "round") else sy_flat
    sz = sz_func(N0)
    ex = ex_func(N0)
    ey = ey_func(N0) if (kind == "round") else ey_flat
    dpp = dpp_func(N0)
    for sym, var in zip(["sx", "sy"],[sx, sy]):
        res+= newval(sym+kind, f"{var:.1f}", r'\micro m')
    for sym, var in zip(["ex", "ey"],[ex*1e3, ey*1e3]):
        res+=newval(sym+kind, f"{var:.1f}", r'nm')
    res+=newval('sz'+kind, f"{sz/10000:.0f}", r'cm')
    res+=newval('st'+kind, f"{sz/1000000/cspeed*1e9:.2f}", r'ns')
    res+=newval('dpp'+kind, f"{dpp:.1e}", '')
print(res)

% Do not edit this file. It is generated by a Python script.
\ProvidesPackage{numericalvalues}
\usepackage{siunitx}


\newcommand{\valvarNerror}{\SI{2.7e+06}{}}
\newcommand{\valnoiseLevel}{\SI{2.4e+08}{}}
\newcommand{\valnoiseLevelV}{\SI{1.0e-07}{V^2}}
\newcommand{\valtheta}{\SI{3.31e-06}{}}
\newcommand{\valthetarms}{\SI{1.82e-03}{}}
\newcommand{\valxi}{\SI{0.31}{}}
\newcommand{\valDx}{\SI{101}{cm}}
\newcommand{\valbetax}{\SI{204}{cm}}
\newcommand{\valbetay}{\SI{98}{cm}}
\newcommand{\valalphax}{\SI{1.25}{}}
\newcommand{\valalphay}{\SI{-0.87}{}}
\newcommand{\valszZeroCur}{\SI{9}{cm}}
\newcommand{\valspZeroCur}{\SI{8.4e-05}{}}
\newcommand{\valwfLen}{\SI{1.5}{ms}}
\newcommand{\valNrev}{\SI{11250}{}}
\newcommand{\valEbeamDimLess}{\SI{96.4}{}}
\newcommand{\valEbeam}{\SI{96.4}{MeV}}
\newcommand{\valKu}{\SI{1.0}{}}
\newcommand{\vallambdaone}{\SI{1.16}{\micro m}}
\newcommand{\valIbeam}{\SI{2.0}{mA}}
\newcommand{\valMround}{\SI{3.8e+06}{}}
\newcommand{\valMflat}{\SI{4.3e+06}{}}
\newcommand{\val

In [20]:
with open("/home/ilobach/FUR_Run2_experiment_paper/numericalvalues.sty", 'w') as f:
    f.write(res)